In [ ]:
Load the dataset into python environment

In [14]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

titanic_df = pd.read_csv('/home/nandu/Downloads/titanic_dataset.csv')  # Replace 'titanic.csv'

In [ ]:
Make ‘PassengerId’ as the index column

In [4]:
titanic_df.set_index('PassengerId', inplace=True)

In [ ]:
Check the basic details of the dataset

In [5]:
print(titanic_df.info())
print(titanic_df.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB
None
         Survived      Pclass         Age       SibSp       Parch        Fare
count  891.000000  891.000000  714.000000  891.000000  891.000000  891.000000
mean     0.383838    2.308642   29.699118    0.523008    0.381594   32.204208
std      0.486592    0.836071   14.526497    1.102743    0.806057   49.693429
min     

In [ ]:
Fill in all the missing values present in all the columns in the dataset

In [7]:
numeric_columns = titanic_df.select_dtypes(include=['number']).columns
imputer_numeric = SimpleImputer(strategy='mean')
titanic_df[numeric_columns] = imputer_numeric.fit_transform(titanic_df[numeric_columns])

# Step 4: Fill in missing values in non-numeric columns
non_numeric_columns = titanic_df.select_dtypes(exclude=['number']).columns
imputer_non_numeric = SimpleImputer(strategy='most_frequent')
titanic_df[non_numeric_columns] = imputer_non_numeric.fit_transform(titanic_df[non_numeric_columns])

In [ ]:
Check and handle outliers in at least 3 columns in the dataset

In [9]:
columns_with_outliers = ['Age', 'Fare', 'SibSp']

for column in columns_with_outliers:
    # Identify and handle outliers using z-score method
    z_scores = (titanic_df[column] - titanic_df[column].mean()) / titanic_df[column].std()
    outliers = (z_scores > 3) | (z_scores < -3)
    titanic_df.loc[outliers, column] = titanic_df[column].median()

In [ ]:
Do min max scaling on the feature set (Take ‘Survived’ as target)

In [15]:
categorical_columns = titanic_df.select_dtypes(include=['object']).columns

# Create a ColumnTransformer to apply different transformations to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ])

# Apply the transformations to the entire dataset
X = preprocessor.fit_transform(titanic_df)

# Target variable
y = titanic_df['Survived']

# Optionally, split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)